This notebook 

In [1]:
#import packages and initialize settings
import os
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii
from reproject import reproject_interp

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)
plt.rcParams['axes.facecolor']='white'
plt.rcParams['savefig.facecolor']="white"

import sys
sys.path.append('/home/mayker.1/Desktop/PythonFunctions')
from SNeGenerationModels import runModels
from FindNearestMC import nearestMCMethod

In [2]:
# Get SNeCO information
database = Table.read('../Data/3.SNe+GalData.csv', format = "csv")

database[0:1]

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits


In [3]:
galaxies = np.array(database["galaxy"], dtype='object')   
SNe, SNra, SNdec  = np.array(database["SN_name"], dtype='object'), np.array(database["SN_ra"], dtype='object'), np.array(database["SN_dec"], dtype='object')
telOrient = np.array(database["telOrient"], dtype='object')
centerCoords = [(database["orient_ra"][i], database["orient_dec"][i]) for i in range(len(database))] 

#fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_1kpc.fits"
fileName = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/ngc0628_irac1_gauss7p5.fits"
hdulist = pyfits.open(fileName)
map = hdulist[0].data
#wcs = WCS(hdulist[0].header, naxis=2)



In [4]:
# generate list of galaxies in IRAC sample and Phangs Alma coverage maps
# "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "_12m+7m+tp_co21_150pc_coverage2d.fits"

IRFileList, PACovFileList = [],[]

for i in range(len(galaxies)):
    PACovFileList.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxies[i] + "/" + galaxies[i] + "_" + telOrient[i] + "_co21_150pc_coverage2d.fits")
    if galaxies[i] == "circinus":
        galaxies[i] = "eso097-013"
    fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/irac/" + galaxies[i] + "_irac1_gauss7p5.fits"
    if os.path.isfile(fileCheck) == False:
        fileCheck = "/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/" + galaxies[i] + "_w1_gauss7p5.fits"
        if os.path.isfile(fileCheck) == False:
            fileCheck = "No IRAC or WISE File for " + galaxies[i] + "."        
    else: pass
    IRFileList.append(fileCheck)

    

In [5]:
# for i in range(len(fileList)):
#     print(fileList[i])

# database.add_column(PACovFileList, name ="PACovFile")
# database.add_column(IRFileList, name = "IFRFile")

database[0:1]

# ascii.write(database, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile,PACovFile,IFRFile
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74,str112,str97
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_coverage2d.fits,/data/tycho/0/leroy.42/projects/phangs_survey_2019/working_data/z0mgs/eso097-013_w1_gauss7p5.fits


Missing IRAC files for Circinus, NGC2997, NGC4945, NGC5128, NGC5530, NGC6744
Wise files available in ZOMGS directory
Missing 150pc res and coverage files for NGC 1068, 1672, & 4579
for now will leave out and maybe revisit with another resolution later?

What resolution is this map: 7p5 arcsec
W1 wise is 3.4 microns.
"IRAC is a four-channel camera that provides simultaneous 5.2 x 5.2 arcmin images at 3.6, 4.5, 5.8, and 8 microns (the black instrument in the lower right part of the above image; see the optical housing model and the conceptual layout diagram. The pixel size is 1.22 arcsec in all bands. "

In [6]:
# remove duplicate galaxies, grouping SNe together in their own lists.

gal_group = database.group_by("galaxy")

SNr, SNd, SNa = [],[],[]
gala, ifrs, covs, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea = [],[],[],[],[],[],[],[],[],[],[],[],[]

for i in range(len(database)):
    gal = database["galaxy"][i]
    ifr = database["IFRFile"][i]
    cov = database["PACovFile"][i]
    img = database["ImageFile150"][i]
    ers = database["ErrFile150"][i]
    aco = database["AlphaCOFile"][i]
    ccs = centerCoords[i]
    pas = database["orient_posang"][i]
    inc = database["orient_incl"][i]
    gds = database["dist"][i]
    SNr.append(SNra[i])
    SNd.append(SNdec[i])
    SNa.append(SNe[i])
    
    if (i+1) in gal_group.groups.indices:
        #print(gal)
        if gal != "ngc1068" and gal != "ngc1672" and gal != "ngc4579":
            #print(gal)
            gala.append(gal)
            ifrs.append(ifr)
            covs.append(cov)
            imgs.append(img)
            errs.append(ers)
            alco.append(aco)
            cocs.append(ccs)
            pans.append(pas)
            incl.append(inc)
            gdis.append(gds)
            SNrs.append(SNr)
            SNdc.append(SNd)
            SNea.append(SNa) 
            SNr, SNd, SNa = [],[],[]
        else: pass
    else: pass

In [7]:
# Next apply clipping to match field of coverage
# use reproject_interp and project IRAC onto the PHANGS one and keep the overlap coverage as the locations that 
# the SNe can occur in 

# gala, ifrs, covs, imgs, errs, alco, cocs, pans, incl, gdis, SNrs, SNdc, SNea



with open("../Data/NearestMC_Stellar.txt", "w") as textfile:
    print("#Galaxy, dist to neares 10^5.5 MC, dist to nearest 10^6.5 MC", file = textfile)
    #for i in range(1):

    for i in range(len(gala)):
        if incl[i] == 90.0:
            incl[i] = 80.0
        print(gala[i])
        
        covFile = covs[i]
        ifrFile = ifrs[i]
        hdu_cov = pyfits.open(covFile)
        hdu_ifr = pyfits.open(ifrFile)
        covMap  = hdu_cov[0].data
        ifrMap, footprint = reproject_interp(hdu_ifr, hdu_cov[0].header)

        #wcs     = WCS(hdu_ifr[0].header, naxis=2)
        wcs     = WCS(hdu_cov[0].header, naxis=2)

        naxis   = wcs._naxis # size of image naxis[0] = x and [1] = y
        grid    = np.indices((naxis[1],naxis[0]))
        ra, dec = wcs.wcs_pix2world(grid[1],grid[0],0) 
        
        f_cov = covMap.flatten()
        f_ifr = ifrMap.flatten()  
        f_ra  = ra.flatten()
        f_dec = dec.flatten()
        
        keep = np.where((f_cov > 0.9) & np.isfinite(f_ifr))
        ifr  = f_ifr[keep]
        ra   = f_ra[keep]
        dec  = f_dec[keep]

        starRandra, starRanddec, starRanddx, starRanddy = runModels(gala[i], imgs[i], cocs[i], pans[i], incl[i], gdis[i], modelType = 3, starLight = ifr, starRa = ra, starDec = dec)

        
        nearest55, nearest65 = nearestMCMethod(gala[i], imgs[i], errs[i], alco[i], cocs[i], pans[i], incl[i], gdis[i], starRandra, starRanddec)
            
#Clean this up        #if np.isnan(nearest55):
        print(gala[i], nearest55, nearest65, file = textfile)




    

circinus
at Mass cutoff A for circinus
Nearest 55 [ 358.17189652  147.75827459    0.            0.            0.
  242.52419152   60.10670105    0.            0.            0.
   60.17176873  363.62918479    0.          242.58855785    0.
  469.85410525    0.          200.01606176    0.           97.00233808
    0.          491.79748757  200.00485626    0.          131.78665084
    0.          124.898752     97.01501538    0.            0.
    0.          633.14456116    0.            0.            0.
    0.            0.           64.99365935    0.            0.
    0.            0.          465.03718332   60.0793636   433.98520296
    0.            0.          485.23686845    0.         1781.60553647
    0.            0.          194.7429385     0.            0.
  131.63724701    0.            0.            0.           64.96694172
  316.65085678  474.98019614    0.            0.          124.48470607
    0.          441.88495404  791.01273814  974.37070488    0.
   60.06432842    0.

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


at Mass cutoff A for ngc0253
Nearest 55 [  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.         230.3845866    0.           0.
   0.           0.           0.           0.           0.
  56.50817463   0.         177.95581467   0.           0.
   0.           0.           0.         133.38712471   0.
   0.           0.           0.           0.           0.
 283.26795178   0.           0.           0.          56.50165267
 416.88745083  98.28733436   0.           0.           0.
   0.         224.09769081   0.           0.           0.
   0.           0.           0.           0.           0.
   0.         230.34392923   0.           0.         520.58578665
   0.         194.44966695   0.           0.           0.
 179.61180692   0.         546.35877643 281.15881426   0.
   0.           0.          56.38412069   0.           0.
   0.           

at Mass cutoff A for ngc1087
Nearest 55 [ 157.33948601  657.58260493    0.          128.19065634  116.47433508
    0.            0.            0.            0.            0.
    0.            0.          237.95420629    0.            0.
    0.            0.            0.            0.            0.
  203.68083072   23.05534612    0.            0.           67.24057824
  487.61528503    0.            0.          297.1203523     0.
    0.          159.24633348  157.33935971   38.82476462   31.46782829
    0.            0.          130.78795773   46.110691      0.
    0.            0.            0.           39.19423047  164.11952791
   62.93564609    0.            0.            0.            0.
   23.05534595  454.27415223    0.            0.          209.56137121
    0.            0.            0.          271.77322636 2578.71300182
 2221.14194914    0.            0.           46.1106935     0.
   69.16604022    0.            0.            0.           62.93579014
   93.97525325    0.  

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
at Mass cutoff A for

...

at Mass cutoff A for ngc3627
Nearest 55 [  0.           0.          80.48515102   0.           0.
   0.           0.           0.          22.33353591  64.71295624
   0.           0.         119.87257216 620.42830406   0.
   0.           0.           0.         184.38264692   0.
   0.         593.11072992   0.           0.           0.
   0.           0.           0.          22.33358865  22.33355036
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
 155.31341739   0.          72.71141707   0.           0.
   0.          92.18845224 227.93343751   0.           0.
   0.           0.           0.           0.           0.
   0.          80.85500902   0.           0.          80.84629543
   0.           0.           0.           0.           0.
   0.           0.         786.38535171  83.42529522   0.
 273.97070043   0.          43.0593774    0.         947.173222
   0.           0.           0.           0.         159.780

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
at Mass cutoff A for

at Mass cutoff A for ngc4457
Nearest 55 [   0.            0.          193.21759395    0.            0.
    0.           31.5250887    32.09738932  343.95504713   22.97568323
  111.98586148   45.95137085    0.            0.            0.
  400.84564918    0.            0.            0.            0.
  174.94538725    0.            0.            0.            0.
    0.            0.            0.          271.68493336    0.
    0.            0.          842.54266983    0.            0.
    0.          380.96536362    0.            0.            0.
    0.            0.            0.           32.09726827    0.
    0.            0.            0.            0.          577.25696772
    0.            0.            0.         1222.48179762    0.
  220.67470221    0.            0.            0.          160.48698652
   45.95137023   88.01386805    0.            0.            0.
    0.          612.0196974     0.            0.            0.
    0.          725.99439039    0.            0.      

at Mass cutoff A for ngc4945
Nearest 55 [   0.           47.80579422    0.          326.27640561    0.
    0.            0.            0.           47.80477704    0.
  239.43072361    0.            0.            0.            0.
  279.79184906    0.            0.           95.70397253  628.26210167
    0.          624.36650487  143.74936635    0.            0.
    0.            0.            0.            0.            0.
    0.          480.04849378  688.03815863    0.          191.21272542
    0.            0.            0.          602.46958428    0.
    0.          324.82476362    0.          284.04545477    0.
 1458.35024154    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.         1120.

at Mass cutoff A for ngc5530
Nearest 55 [  65.1419461   283.72568286   65.1335755    83.01101633    0.
  129.67417731    0.            0.            0.           26.43957417
  360.81754291    0.            0.          410.79470854  335.74638864
  179.78515877    0.          128.61002218    0.           52.99852388
    0.         1236.10646189  180.40655487   26.44288416   26.44009851
    0.           89.33226882    0.            0.           84.11162337
   77.10454329    0.            0.          356.02779125  389.10599316
  102.72619494    0.            0.            0.          187.45965995
    0.          690.89937979   41.50919501  270.28978911    0.
    0.          127.94288817  205.44648949    0.           79.00768579
    0.          231.54716049    0.            0.          304.91958582
    0.           41.52249332    0.            0.           89.06891771
  153.84478496    0.            0.            0.         1281.4361623
    0.            0.          767.02410669    0.      

In [8]:
print(nearestMCs)

NameError: name 'nearestMCs' is not defined

In [ ]:
#from FindNearestMC import findNearest

# def findNearest(x1,x2,y1,y2):
#     return(mindist) 

# if this uses too much memory, play minesweeper, 0 for gas and grow
# scipy scikit-image

# or monte carlo and only do numpy random choice to pick weigheted probability choice from the probability arrays 
# and then do like 10,000 draws from that array before feeding into the pair_dist_calc


# can imshow the matricies 
# print the shapes of the tile columns, etc
